<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/explorations1/explore_nbs/collab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200707" #"nightly"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py
!pip install https://github.com/butchland/fastai_xla_extensions/archive/master.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  23250      0 --:--:-- --:--:-- --:--:-- 23250
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Done updating TPU runtime
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     


In [2]:
import fastai_xla_extensions 

In [3]:
!pip install https://github.com/fastai/fastcore/archive/master.zip
!pip install https://github.com/fastai/fastai2/archive/master.zip

     - 3.6MB 4.0MB/s
  Created wheel for fastcore: filename=fastcore-0.1.18-cp36-none-any.whl size=28891 sha256=36464b0c83da0be54c7276c9ed26a1eeadc151310bb9ce1c553bef5ed93d6fac
  Stored in directory: /tmp/pip-ephem-wheel-cache-rln3ddii/wheels/68/af/c5/c8a9f7370515ab9b237b3fd59c04a0e7c5d4dedc7a6768a772
Successfully built fastcore
     / 121.2MB 1.1MB/s
  Created wheel for fastai2: filename=fastai2-0.0.18-cp36-none-any.whl size=175594 sha256=e0074668632763d927675a79338ed91df8c58bfac2461714218543b365e1eeb7
  Stored in directory: /tmp/pip-ephem-wheel-cache-k48hc1u4/wheels/b2/28/3b/7a38644c6129e851e0db09b594949028266a62f4cb234fd1f3
Successfully built fastai2
  Found existing installation: fastai2 0.0.17
    Uninstalling fastai2-0.0.17:
      Successfully uninstalled fastai2-0.0.17


In [4]:
from fastai2.vision.all import *

[('__call__', <function LevelMapper.__call__ at 0x7f1e4885b840>), ('__init__', <function LevelMapper.__init__ at 0x7f1e4885b7b8>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7f1e4879d400>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7f1e4879d378>)]
[('__init__', <function BoxCoder.__init__ at 0x7f1e48728840>), ('decode', <function BoxCoder.decode at 0x7f1e487289d8>), ('decode_single', <function BoxCoder.decode_single at 0x7f1e48728a60>), ('encode', <function BoxCoder.encode at 0x7f1e487288c8>), ('encode_single', <function BoxCoder.encode_single at 0x7f1e48728950>)]
[('__call__', <function Matcher.__call__ at 0x7f1e48728730>), ('__init__', <function Matcher.__init__ at 0x7f1e48728b70>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7f1e487287b8>)]
[('__init__', <function ImageList.__init__ at 0x7f1e48728d08>), ('to', <function ImageList.to at 0x7f1e48728c80>)]
[('__init__', <function Timebase.__init__ at

In [5]:
path = untar_data(URLs.MNIST_TINY)
datablock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW),CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(),
    item_tfms=Resize(28),
    batch_tfms=[]
)
#datablock.summary(path)

In [6]:
class Lenet2(nn.Module):
    """Lenet with layers"""
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3) # set 3 for first item if RGB
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) # Only 2 outputs (3 and 7) instead of 10
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
# Use same datablock, but it goes to TPU because importing first and patched default device
dls_tpu = datablock.dataloaders(path)

In [8]:
learner = Learner(dls_tpu, Lenet2(), metrics=accuracy, opt_func=Adam)

In [9]:
%load_ext tensorboard

In [10]:
from torch.utils.tensorboard import SummaryWriter

#export
class TensorBoardCallback(Callback):
    "Saves model topology, losses & metrics"
    def __init__(self, log_dir=None, trace_model=True, log_preds=True, n_preds=9):
        store_attr(self, 'log_dir,trace_model,log_preds,n_preds')

    def begin_fit(self):
        self.run = not hasattr(self.learn, 'lr_finder') and not hasattr(self, "gather_preds") and rank_distrib()==0
#        print(f"writing to {self.log_dir}")
        self.writer = SummaryWriter(log_dir=self.log_dir)
        if self.trace_model:
            if hasattr(self.learn, 'mixed_precision'):
                raise Exception("Can't trace model in mixed precision, pass `trace_model=False` or don't use FP16.")
            b = self.dls.one_batch()
            self.learn._split(b)
            self.writer.add_graph(self.model, *self.xb)

    def after_batch(self):
        self.writer.add_scalar('train_loss', self.smooth_loss, self.train_iter)
        for i,h in enumerate(self.opt.hypers):
            for k,v in h.items(): self.writer.add_scalar(f'{k}_{i}', v, self.train_iter)

    def after_epoch(self):
        for n,v in zip(self.recorder.metric_names[2:-1], self.recorder.log[2:-1]):
            self.writer.add_scalar(n, v, self.train_iter)
        if self.log_preds:
            b = self.dls.valid.one_batch()
            self.learn.one_batch(0, b)
            preds = getattr(self.loss_func, 'activation', noop)(self.pred)
            out = getattr(self.loss_func, 'decodes', noop)(preds)
            x,y,its,outs = self.dls.valid.show_results(b, out, show=False, max_n=self.n_preds)
            tensorboard_log(x, y, its, outs, self.writer, self.train_iter)

    def after_fit(self): self.writer.close()


#export
@typedispatch
def tensorboard_log(x:TensorImage, y: TensorCategory, samples, outs, writer, step):
    fig,axs = get_grid(len(samples), add_vert=1, return_fig=True)
    for i in range(2):
        axs = [b.show(ctx=c) for b,c in zip(samples.itemgot(i),axs)]
    axs = [r.show(ctx=c, color='green' if b==r else 'red')
            for b,r,c in zip(samples.itemgot(1),outs.itemgot(0),axs)]
    writer.add_figure('Sample results', fig, step)

@typedispatch
def tensorboard_log(x:TensorImage, y: (TensorImageBase, TensorPoint, TensorBBox), samples, outs, writer, step):
    fig,axs = get_grid(len(samples), add_vert=1, return_fig=True, double=True)
    for i in range(2):
        axs[::2] = [b.show(ctx=c) for b,c in zip(samples.itemgot(i),axs[::2])]
    for x in [samples,outs]:
        axs[1::2] = [b.show(ctx=c) for b,c in zip(x.itemgot(0),axs[1::2])]
    writer.add_figure('Sample results', fig, step)


In [11]:
learner.fit(10, 10e-3, cbs=TensorBoardCallback(Path("/temp/runs"))) # 0.05) NOTE: Im not sure if this works...!!! it is now 96!

epoch,train_loss,valid_loss,accuracy,time
0,0.393893,0.077436,0.978541,00:01
1,0.241319,0.036550,0.992847,00:02
2,0.166562,0.018606,0.995708,00:01
3,0.125944,0.020083,0.991416,00:01
4,0.095399,0.013837,0.992847,00:01
5,0.074669,0.015045,0.995708,00:01
6,0.061635,0.013224,0.992847,00:01
7,0.050614,0.018656,0.994278,00:02
8,0.041156,0.010657,0.997139,00:01
9,0.033956,0.008123,0.997139,00:01


In [15]:
%tensorboard --logdir /temp

Reusing TensorBoard on port 6006 (pid 1025), started 0:04:54 ago. (Use '!kill 1025' to kill it.)

<IPython.core.display.Javascript object>